In [1]:
%matplotlib inline
from os import listdir, path, remove
from requests import get
import urllib3
from lxml import etree
import pandas as pd
import warnings
from tqdm import tqdm
import itertools
warnings.filterwarnings('ignore')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
#Data Rutte2
for year in tqdm(range(2011, 2018)):
    endOfAssembly = False
    assemblyNr = 0
    while endOfAssembly == False:
        assemblyNr += 1
        for debateNr in range(1, 200):
            file = "h-tk-%s-%s-%s" %(str(year)+str(year+1), str(assemblyNr), str(debateNr))
            if not path.isfile('HandelingenTK/%s.xml' %file) or not path.isfile('HandelingenTKmeta/%smetadata.xml' %file):
                url = "https://zoek.officielebekendmakingen.nl/%s" %file
                resp = get(url + ".xml", verify=False, allow_redirects=False)
                if resp.status_code == 200:
                    # Download the Handelingen file
                    hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
                    hand_file.write(resp.content)
                    hand_file.close()
                    resp = get(url+"/metadata.xml", verify=False, \
                                                        allow_redirects=False)
                    # Download the corresponding meta file
                    meta_file = open('HandelingenTKmeta/%smetadata.xml' %file, 'wb')
                    meta_file.write(resp.content)
                    meta_file.close()    
                elif debateNr == 1:
                    endOfAssembly = True
                    break
                else:
                    break

In [ ]:
#Data Balkenende4
for year in tqdm(range(2006, 2011)):
    for i in range(1,10000):
        for j in range(i,i+100):
            file = "h-tk-%s-%s-%s" %(str(year)+str(year+1), str(i), str(j))
            if not path.isfile('HandelingenTK/%s.xml' %file) or not path.isfile('HandelingenTKmeta/%smetadata.xml' %file):
                url = "https://zoek.officielebekendmakingen.nl/%s" %file
                resp = get(url + ".xml", verify=False, allow_redirects=False)
                if resp.status_code == 200:
                    # Download the Handelingen file
                    hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
                    hand_file.write(resp.content)
                    hand_file.close()
                    resp = get(url+"/metadata.xml", verify=False, \
                                                        allow_redirects=False)
                    # Download the corresponding meta file
                    meta_file = open('HandelingenTKmeta/%smetadata.xml' %file, 'wb')
                    meta_file.write(resp.content)
                    meta_file.close()    

In [2]:
enddict = {}
index = 0
for file in listdir('HandelingenTK/'):
    context = etree.iterparse('HandelingenTK/' + file,  tag='spreekbeurt')
    mainSpeaker = None
    old = True
    for _, elem in context:
        old = False
        if elem.get('nieuw') == 'ja':
            MainSpeechnumber = index
            speechCategory = "Main Speech"
            mainSpeaker = elem.findtext('spreker/naam/achternaam')
        elif elem.findtext('spreker/naam/achternaam') == mainSpeaker:
            speechCategory = "Response"
            enddict[MainSpeechnumber][2] += " " + ' '.join(elem.find('tekst').itertext())
            continue
        else:
            speechCategory = "Interruption"
        surname = elem.findtext('spreker/naam/achternaam')
        party = elem.findtext('spreker/politiek')
        text = ' '.join(elem.find('tekst').itertext())
        enddict[index] = [surname, party, text, file, speechCategory]
        del surname, party, text, speechCategory
        index += 1
        
    if old:
        context = etree.iterparse('HandelingenTK/' + file,  tag='spreker')
        mainSpeaker = None
        for _, elem in context:
            if elem.get('nw') == 'ja':
                MainSpeechnumber = index
                speechCategory = "Main Speech"
                mainSpeaker = elem.findtext('wie/naam')
            elif elem.findtext('wie/naam') == mainSpeaker:
                speechCategory = "Response"
                enddict[MainSpeechnumber][2] += " "+' '.join([x.text for x in elem.findall('al')])
                continue
            else:
                speechCategory = "Interruption"
            surname = elem.findtext('wie/naam')
            party = elem.findtext('wie/partij')
            text = ' '.join([x.text for x in elem.findall('al')])
            enddict[index] = [surname, party, text, file, speechCategory]
            del surname, party, text, speechCategory
            index += 1

    
df = pd.DataFrame.from_dict(enddict, orient='index')
df = df.rename(columns={0:'surname', 1:'party', 2:'text', 3:'file', 4:'speech category'})
df.head(5)

,surname,party,text,file,speech category
0,Zalm,None,Mevrouw de voorzitter! Het is mij een genoegen...,h-tk-19981999-1-2.xml,Main Speech
1,Zalm,None,Mevrouw de voorzitter! Ik was nog niet geboren...,h-tk-20002001-1-2.xml,Main Speech
2,plaatsvervangend griffier van de Tweede Kamer,None,De Koningin!,h-tk-20012002-1-1.xml,Interruption
3,Zalm,None,Mevrouw de voorzitter. Het is anders dan ander...,h-tk-20012002-1-2.xml,Main Speech
4,Hoogervorst,None,Mijnheer de voorzitter. De eerste miljoenennot...,h-tk-20022003-1-2.xml,Main Speech


In [3]:
metaDict = {}
for file in listdir('HandelingenTKmeta'):
    filet = str(open('HandelingenTKmeta/' + file, 'rb').read())
    if "HTTP 404." in filet or "502 Bad Gateway" in filet or "Er is een probleem met de pagina die u zoekt, en kan niet worden getoond." in filet:
        remove('HandelingenTKmeta/' + file)
    else:
        tags = []
        date = None
        context = etree.iterparse('HandelingenTKmeta/' + file,  tag='metadata')
        for _, elem in context:
            if elem.get("name") == "OVERHEIDop.datumVergadering":
                date = elem.get('content')
            elif elem.get('name') == "OVERHEID.category":
                tags.append(elem.get('content'))
            metaDict[file[:-12] + '.xml'] = [date, tags]
metaDF = pd.DataFrame.from_dict(metaDict, orient='index')
metaDF = metaDF.rename(columns={0:'date', 1:'tags'})
metaDF.head(5)

,date,tags
h-tk-20062007-1-1.xml,2006-09-19,[Bestuur | Parlement]
h-tk-20062007-1-3.xml,2006-09-19,"[Financiën | Organisatie en beleid, Financiën ..."
h-tk-20062007-1-4.xml,2006-09-19,[Bestuur | Parlement]
h-tk-20062007-10-10.xml,2006-09-20,"[Bestuur | Provincies, Bestuur | Gemeenten]"
h-tk-20062007-10-11.xml,2006-09-20,"[Openbare orde en veiligheid | Criminaliteit, ..."


In [4]:
combineddf = df.join(metaDF, on='file')
combineddf = combineddf.dropna()
#https://stackoverflow.com/questions/29370057/select-dataframe-rows-between-two-dates
combineddf['date'] = pd.to_datetime(combineddf['date'])

In [5]:
rutte2DF = combineddf
rutte2DF = rutte2DF.loc[(rutte2DF['date'] > '2012-11-05') & (rutte2DF['date'] <= '2017-03-23')]
rutte2DF = rutte2DF[rutte2DF['party'].isin(['50PLUS', 'CDA', 'ChristenUnie', 'D66', 'GroenLinks', 'PVV', 'PvdA', 'PvdD', 'SGP', 'SP', 'VVD'])]
rutte2DF.to_csv("HandelingenRutte2.csv")
rutte2DF.head(5)

,surname,party,text,file,speech category,date,tags
175881,Wilders,PVV,\n \n Mevrouw de voorzit...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"
175882,Roemer,SP,\n \n Voorzitter. Vorige...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"
175883,Pechtold,D66,\n \n Voorzitter. Twee m...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"
175884,Van Haersma Buma,CDA,\n \n Voorzitter. Alleen...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"
175886,Wilders,PVV,\n \n Duidelijkheid en e...,h-tk-20122013-100-3.xml,Interruption,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"


In [6]:
balkenende4DF = combineddf
balkenende4DF = balkenende4DF.loc[(balkenende4DF['date'] > '2007-02-22') & (balkenende4DF['date'] <= '2010-02-20')]
balkenende4DF = balkenende4DF[balkenende4DF['party'].isin(['CDA', 'ChristenUnie', 'D66', 'GroenLinks', 'PVV', 'PvdA', 'PvdD', 'SGP', 'SP', 'VVD'])]
balkenende4DF.to_csv("HandelingenBalkenende4.csv")
balkenende4DF.head(5)

,surname,party,text,file,speech category,date,tags
5119,Marijnissen,SP,Voorzitter. Wij spreken vandaag voor het eerst...,h-tk-20062007-2634-2731.xml,Main Speech,2007-03-01,[Bestuur | Organisatie en beleid]
5120,Tichelaar,PvdA,Voorzitter. Ik wil graag weten wat de heer Mar...,h-tk-20062007-2634-2731.xml,Interruption,2007-03-01,[Bestuur | Organisatie en beleid]
5121,Tichelaar,PvdA,Ik refereerde aan de opmerking van de heer Mar...,h-tk-20062007-2634-2731.xml,Interruption,2007-03-01,[Bestuur | Organisatie en beleid]
5122,Tichelaar,PvdA,Ik ga absoluut in op het laatste. De komende v...,h-tk-20062007-2634-2731.xml,Interruption,2007-03-01,[Bestuur | Organisatie en beleid]
5123,Tichelaar,PvdA,Hij moest daar als een kleuter staan en de per...,h-tk-20062007-2634-2731.xml,Interruption,2007-03-01,[Bestuur | Organisatie en beleid]
